In [7]:
import pandas as pd
import numpy as np
from os.path import join
from datetime import datetime, timedelta

In [8]:
REC_P = 0.2

In [9]:
def generate_batch(batch_id, start_datetime):
    '''generates several records of format TagName/DateTime/Value/StringValue'''
    t = start_datetime
    b = 'B{}'.format(10000+batch_id)
    data = []
    # record batch on 1st equipment
    data.append(['EQ01.BATCH', str(t), np.nan, b])
    # record status 0 on 1st equipment
    data.append(['EQ01.STATUS', str(t), 0, 0])
    # generate a random time for the 1st equipment
    t1 = 1800+np.random.randint(120)-60
    # record signals
    for i in range(t1):
        # record value
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ01.POWER', str(t), value, value])
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ01.TEMP', str(t), value, value])
        t += timedelta(seconds=1)
    t1_final = t
    # record batch on 2nd equipment
    data.append(['EQ02.BATCH', str(t), np.nan, b])
    # record status 0 on 2nd equipment
    data.append(['EQ02.STATUS', str(t), 0, 0])
    # generate a random time for the 1st equipment
    t20 = 2400+np.random.randint(120)-60
    # record signals
    for i in range(t20):
        # record value
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ02.POWER', str(t), value, value])
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ02.TEMP', str(t), value, value])
        t += timedelta(seconds=1)
    # record status 1 on 2nd equipment
    data.append(['EQ02.STATUS', str(t), 1, 1])
    # generate a random time for the 1st equipment
    t21 = 3000+np.random.randint(120)-60
    # record signals
    for i in range(t21):
        # record value
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ02.POWER', str(t), value, value])
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ02.TEMP', str(t), value, value])
        t += timedelta(seconds=1)
    # record status 2 on 2nd equipment
    data.append(['EQ02.STATUS', str(t), 2, 2])
    # generate a random time for the 1st equipment
    t22 = 1200+np.random.randint(120)-60
    # record signals
    for i in range(t22):
        # record value
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ02.POWER', str(t), value, value])
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ02.TEMP', str(t), value, value])
        t += timedelta(seconds=1)
    # record batch on 3rd equipment
    data.append(['EQ03.BATCH', str(t), np.nan, b])
    # record status 0 on 3rd equipment
    data.append(['EQ03.STATUS', str(t), 0, 0])
    # generate a random time for the 1st equipment
    t3 = 600+np.random.randint(120)-60
    # record signals
    for i in range(t3):
        # record value
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ03.POWER', str(t), value, value])
        if np.random.binomial(1, p=REC_P):
            value = np.random.randint(10)
            data.append(['EQ03.TEMP', str(t), value, value])
        t += timedelta(seconds=1) 
    return data, t1_final, t

In [10]:
def split(batch_data):
    this_day = []
    next_day = []
    for x in (batch_data):
        if datetime.strptime(x[1], "%Y-%m-%d %H:%M:%S").hour > 12:
            this_day.append(x)
        else:
            next_day.append(x)
    return this_day, next_day

In [12]:
start_datetime = datetime.strptime("2019-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")
t = start_datetime
day_data = []

for batch_id in range(1, 500):
    # generate batch
    batch_data, t1_final, t = generate_batch(batch_id, t)
    # check if batch goes through 2 days
    if (t-start_datetime).days > 0:
        this_day, next_day = split(batch_data)
        day_data.extend(this_day)
        # save data
        df = pd.DataFrame(data=day_data, columns=['TagName', 'DateTime', 'Value', 'StringValue'])
        df.to_csv(join('..', 'data', str(start_datetime)[:10] + '.csv'), index=False)
        # start new day
        day_data = next_day
        start_datetime += timedelta(seconds=24*60*60)
    else:
        # add batch data
        day_data.extend(batch_data)
    t = t1_final + timedelta(seconds=1800)